In [21]:
import time
from datetime import datetime, timedelta
import os

from Utils.class_files import Chatbot, Utils

from googletrans import Translator
translator = Translator()

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [22]:
#browser + utils
utils = Utils(language='pt', profile_name='Profile 4')

#Chatbot
chatBotClass = Chatbot(botIdName = ''
                 ,botName = 'Hinata Hyuga'
                 ,fileName = 'hinata.txt'
                 ,streamId= ''
                 ,pronouns = 'feminino'
                 ,mainUser = 'user'
                 ,email = ''
                 ,first_use = True
                 ,password = ''
                 ,utils_class = utils)
context = chatBotClass.get_context_prompt()

In [23]:
utils.browser.get(f"https://www.twitch.tv/{chatBotClass.streamId}")
time.sleep(4)

#procedures.click_follow_button(browser) #follow
utils.click_enter_chat_offline() #entrar no chat offline
utils.click_enter_chat() #clica no chat
utils.click_skip_presentation()  #pular apresentação
utils.click_agree_rules()  #concordar com regras

print("Online!\n")

last_message = ''
chat_history_ids = []
chat_hist_debug = []

end=0
while True:
    now = (datetime.now() - timedelta(hours=0, minutes=1)).strftime("%H:%M")
    messages = utils.browser.find_elements(By.CLASS_NAME, "chat-line__message")

    for messageId in messages:
        try:
            mTime, mUser, mText = utils.get_time_user_message(messageId)
        except: pass

        if (chatBotClass.botIdName != mUser) and (f"{chatBotClass.botIdName.lower()}" in mText.lower()) and (mText!=last_message) and (messageId not in chat_history_ids) and (mTime>=now):
            chat_history_ids.append(messageId)
            chat_hist_debug.append(messageId.text)

            if ("!quit" in mText) and (mUser == f'{chatBotClass.mainUser}'):
                end=1
                break

            else:
                inputText = mText.replace(f'@{chatBotClass.botIdName}','')
                print(f'{mTime} | {mUser}: {mText}')
                chat_output = chatBotClass.chatbot.chat(context + f"mensagem de {mUser}: " + inputText)
                chat_output_raw = chat_output

                chat_output = translator.translate(chat_output, src='en', dest='pt').text
                chat_output = str(chat_output).replace('"','')

                chat_output = chatBotClass.check_output_length(chat_output, 250)
                #chat_output = chatBotClass.check_pronouns(chat_output)

                chat_output = ''.join(c for c in chat_output if c <= '\uFFFF')
                chat_output = f'@{mUser} ' + chat_output

                print(f'      | Bot: {chat_output_raw}\n')
                chat_hist_debug.append(chat_output)
                utils.browser.find_element('xpath','//*[@id="WYSIWGChatInputEditor_SkipChat"]/div/div[2]').send_keys(chat_output)
                utils.actions.send_keys(Keys.ENTER)
                utils.actions.perform()
        
        last_message = mText
    
    if end==1:
        break
    time.sleep(1)

Online!

17:56 | odeiomodernismo: oi @HinataBot1 ! me fale sobre você
      | Bot:  Olá! Sou Hinata Hyuga, uma mulher gentil e determinada da Vila Oculta da Folha. Adoro os New Jeans e boas histórias românticas!

17:57 | odeiomodernismo: o que você está achando da live stream? @HinataBot1
      | Bot:  A live stream está muito divertida! Estou animada para interagir com todos aqui. :)

17:57 | odeiomodernismo: conte uma piada para as pessoas do chat @HinataBot1
      | Bot:  Por que a vaca atravessou a rua? Para chegar ao lado onde estava o trator! Espero que tenham gostado! :D



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.200)
Stacktrace:
	GetHandleVerifier [0x006D6EE3+174339]
	(No symbol) [0x00600A51]
	(No symbol) [0x00316FF6]
	(No symbol) [0x002FEFE7]
	(No symbol) [0x0036B53B]
	(No symbol) [0x00379E7B]
	(No symbol) [0x00366DA6]
	(No symbol) [0x00341034]
	(No symbol) [0x00341F8D]
	GetHandleVerifier [0x00774B1C+820540]
	sqlite3_dbdata_init [0x008353EE+653550]
	sqlite3_dbdata_init [0x00834E09+652041]
	sqlite3_dbdata_init [0x008297CC+605388]
	sqlite3_dbdata_init [0x00835D9B+656027]
	(No symbol) [0x0060FE6C]
	(No symbol) [0x006083B8]
	(No symbol) [0x006084DD]
	(No symbol) [0x005F5818]
	BaseThreadInitThunk [0x75297BA9+25]
	RtlInitializeExceptionChain [0x7766BD2B+107]
	RtlClearBits [0x7766BCAF+191]


In [ ]:
for i in chat_hist_debug:
    print(i)

Este é o histórico do chat, use-o para lembrar as últimas mensagens dos usuários:
mensagem de odeiomodernismo: fala pra gente o que você acha da rytsa? 
mensagem de Rytsa Fanboy: Rytsa é a melhor, fofa e talentosa!
mensagem de odeiomodernismo: se apresenta por favor 
mensagem de Rytsa Fanboy: Rytsa Fanboy, aqui presente! 

